# Web Secrapping from the websites present in excel file and exporting it to another excel file after doing some analysis

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import os
import nltk
import string
import re
import nltk
import newspaper
from newspaper import Article
from pathlib import Path
from newspaper import Config
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

In [55]:
# Reading the dataset
df = pd.read_csv("Input.csv")
df.head()

,URL_ID,URL
0,1,https://insights.blackcoffer.com/how-is-login-...
1,2,https://insights.blackcoffer.com/how-does-ai-h...
2,3,https://insights.blackcoffer.com/ai-and-its-im...
3,4,https://insights.blackcoffer.com/how-do-deep-l...
4,5,https://insights.blackcoffer.com/how-artificia...


In [23]:
df.tail()

,URL_ID,URL
165,167,https://insights.blackcoffer.com/role-big-data...
166,168,https://insights.blackcoffer.com/sales-forecas...
167,169,https://insights.blackcoffer.com/detect-data-e...
168,170,https://insights.blackcoffer.com/data-exfiltra...
169,171,https://insights.blackcoffer.com/impacts-of-co...


In [56]:
# Dropping NA values
df=df.dropna(axis=1)

In [5]:
# Defining a function of "remove punctuation" to remove the punctuations
def remove_punc(dmf):
    global nopunc
    nopunc = [char for char in dmf if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return nopunc

In [6]:
# Defining a function of "remove stopwords" to remove the stopwords
def remove_stpwd(nopunc):
    global clean_mess
    clean_mess = [word for word in nopunc.split() if word.lower() not in stpwrd]
    clean_mess=' '.join(clean_mess)
    return clean_mess

In [7]:
# Defining a function to "tokenize" to tokenize the words
def tokenize(clean_mess):
    global tokens
    tokens = word_tokenize(clean_mess.upper())
    
    return tokens

# 1. NEGATIVE SCORE

In [8]:
# Defining a neg_words() function to count the negative words present in the text
def neg_words(tokens):
    global num_neg
    file = open('negdict.txt', 'r')
    neg_words = file.read().split()
    
    num_neg=0
    for n in range(len(tokens)):
        words=str(tokens[n])
        if words in neg_words:
            num_neg=num_neg+1
    return num_neg

# 2. POSITIVE SCORE

In [9]:
# Defining a pos_words() function to count the positive words present in the text
def pos_words(tokens):
    global num_pos
    file = open('posdict.txt', 'r')
    pos_words = file.read().split()
    num_pos=0
    for n in range(len(tokens)):
        words=str(tokens[n])
        if words in pos_words:
            num_pos=num_pos+1
    return num_pos

# 3. POLARITY SCORE

In [10]:
# Defining a function to calculate the "Polarity Score"
def polarity_sc(num_pos, num_neg):
    global polarity
    polarity=(num_pos-num_neg)/((num_pos+num_neg)+0.000001)
    return polarity

# 4. SUBJECTIVITY SCORE

In [11]:
# Defining a function to calculate the "Subjectivity Score"
def subjectivity_sc(num_pos, num_neg,tokens):
    global subjectivity
    subjectivity=(num_pos+num_neg)/(len(tokens)+0.000001)
    return subjectivity

# 5. AVG SENTENCE LENGTH

In [12]:
# Defining a function to calculate the "Average Sentence Length"
def avgsent_len(dmf):
    global avg_sent_len
    text = dmf
    sent=sent_tokenize(dmf)
    avg_sent_len=(len(dmf.split()))/(len(sent))
    return avg_sent_len

# 6. PERCENTAGE OF COMPLEX WORDS

In [13]:
# Defining a function to calculate the "Percentage of Complex Words"
def complex_words_percent(dmf):
    global complex_wrd
    global complex_per
    complex_wrd=0
    wrd=dmf.split()
    for i in wrd:
        if len(i)>2:
            complex_wrd=complex_wrd+1
    complex_per=complex_wrd/len(wrd)
    return complex_per

# 7. FOG INDEX

In [14]:
# Defining a function to calculate the "fog_index"
def fog_index(avg_sent_len,complex_per):
    global fogind
    fogind=0.4*(avg_sent_len+complex_per)
    return fogind

# 8. AVG NUMBER OF WORDS PER SENTENCE

In [15]:
# Defining a function to calculate the "Average no. of words per sentence"
def avg_wrds_in_sent(dmf):
    global avg_wrds_per_sent
    totwrd=len(dmf.split())
    totsent=(len(sent_tokenize(dmf)))
    avg_wrds_per_sent=totwrd/totsent
             
    return avg_wrds_per_sent

# 9. COMPLEX WORD COUNT

In [16]:
# Defining a function to count the "Complex words"
def complex_wrd_count(dmf):
    global complex_wrds
    complex_wrds=0
    words=dmf.split()
    for i in words:
        if len(i)>2:
            complex_wrds=complex_wrds+1
    return complex_wrds

# 10. WORD COUNT

In [17]:
# Defining a function to count the "Total Cleaned Words"
def word_count(clean_mess):
    global wrd_cnt
    wrd_cnt=len((clean_mess.split()))
    return wrd_cnt

# 11. SYLLABLE PER WORD

In [18]:
# Defining a function to count the "No. of Syllables" present in the text
def syllable_count(dmf):
    global syl_count
    text = dmf
    syl_count = 0
    vowels = "aeiou"
    for word in text.lower().split():
        #print(word)
        if word[0] in vowels:
            syl_count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                syl_count += 1
        if word.endswith("ed") or word.endswith("es"):
            syl_count -= 1
    return syl_count

# 12. PERSONAL PRONOUNS

In [19]:
# Defining a function to calculate the "Personal Pronouns"
def personal_pronoun(dmf):
    global personal_pro
    pronounRegex = re.compile(r'\bI\b|\bwe\b|\bWe\b|\bmy\b|\bMy\b|\bours\b|\bus\b')
    pps=pronounRegex.findall(dmf)
    personal_pro=len(pps)
    return personal_pro

# 13. AVG WORD LENGTH

In [20]:
# Defining a function to calculate the "Average Words Length"
def avg_wrd_length(dmf):
    global avg_wrdlen
    alltxt=dmf
    wrd_cnt=0
    char_cnt=0
    for word in alltxt:
        if word!= ' ':
            char_cnt=wrd_cnt+1
    totalchar=char_cnt
    for word in alltxt.split():
        wrd_cnt=wrd_cnt+1
    totalwrds=wrd_cnt
    avg_wrdlen=totalchar/totalwrds
    return avg_wrdlen

In [21]:
# The following code is necessary to increase the time out to prevent "read timed out" error
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'
config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 40

In [25]:
# The following for loop parses through each url in the csv file, downloads the title and the body
# Then it finally appends the text in text file
for n in range(len(df)):
    urlm=df['URL'][n]
    article=Article(urlm, config=config)
    article.download()
    article.parse()
    art=article.title
    art2=article.text
    p = Path(str(df['URL_ID'][n])+'.txt')
    p=  open(str(df['URL_ID'][n])+'.txt', 'a',encoding='utf-8')   
    p.write(art)
    p = open(str(df['URL_ID'][n])+'.txt', 'a',encoding='utf-8')
    p.write('\n \n')
    p.write(art2)
    p.close()

In [26]:
stpwrd = nltk.corpus.stopwords.words('english')

In [27]:
file2=open('StopWords_Auditor.txt')
file2=file2.read().split('\n')

In [28]:
# We can add additional stopwords to the corpus using .extend() method as follows
stpwrd.extend(file2)

In [29]:
mst_dict=pd.read_csv('LoughranMcDonald_MasterDictionary_2020.csv')

In [30]:
# I have created a positive word and a negative word dictionary using the positive and negative score
# from the mst_dict respectively
mst_dict['neg_wrd'] = mst_dict.loc[mst_dict['Negative'] == 2009]['Word']
mst_dict['pos_wrd'] = mst_dict.loc[mst_dict['Positive'] == 2009]['Word']

In [31]:
for n in range(len(mst_dict['neg_wrd'])):
    word=str(mst_dict['neg_wrd'][n])
    if word!='nan':
        p=open('negdict.txt','a')
        p.write(word +' ')

for n in range(86531):
    word=str(mst_dict['pos_wrd'][n])
    if word!='nan':
        p=open('posdict.txt','a')
        p.write(word +' ')

In [53]:
# Now creating a dictionary that will contain all the extracted values of the input file
alldata={ 'URL_ID':[],
          'URL':[],
          'POSITIVE SCORE':[],
          'NEGATIVE SCORE':[],
          'POLARITY SCORE':[],
          'SUBJECTIVITY SCORE':[],
          'AVG SENTENCE LENGTH':[],
          'PERCENTAGE OF COMPLEX WORDS':[],
          'FOG INDEX':[],
          'AVG NUMBER OF WORDS PER SENTENCE':[],
          'COMPLEX WORD COUNT':[],
          'WORD COUNT':[],
          'SYLLABLE PER WORD':[],
          'PERSONAL PRONOUNS':[],
          'AVG WORD LENGTH':[]
        }

In [58]:
# Finally I am writing the main function
for n in range(len(df)):
    file=open(str(df['URL_ID'][n])+'.txt', 'r',encoding='utf-8')
    dmf=file.read()
    l=df['URL'][n]
    alldata['URL'].append(l)
    remove_punc(dmf)
    remove_stpwd(nopunc)
    tokenize( clean_mess)
    neg_words(tokens)
    pos_words(tokens)
    polarity_sc(num_pos, num_neg)
    subjectivity_sc(num_pos, num_neg,tokens)
    avgsent_len(dmf)
    complex_words_percent(dmf)
    fog_index(avg_sent_len,complex_per)
    avg_wrds_in_sent(dmf)
    complex_wrd_count(dmf)
    word_count(clean_mess)
    syllable_count(dmf)
    personal_pronoun(dmf)
    avg_wrd_length(dmf) 
    alldata['URL_ID'].append(n)   
    alldata['NEGATIVE SCORE'].append(num_neg)
    alldata['POSITIVE SCORE'].append(num_pos)
    alldata['POLARITY SCORE'].append(polarity)
    alldata['SUBJECTIVITY SCORE'].append(subjectivity)
    alldata['AVG SENTENCE LENGTH'].append(avg_sent_len)
    alldata['PERCENTAGE OF COMPLEX WORDS'].append(complex_per)
    alldata['FOG INDEX'].append(fogind)
    alldata['AVG NUMBER OF WORDS PER SENTENCE'].append(avg_wrds_per_sent)
    alldata['COMPLEX WORD COUNT'].append(complex_wrds)
    alldata['WORD COUNT'].append(wrd_cnt)
    alldata['SYLLABLE PER WORD'].append(syl_count)
    alldata['PERSONAL PRONOUNS'].append(personal_pro)
    alldata['AVG WORD LENGTH'].append(avg_wrdlen)

In [59]:
# To export the data into excel file, I have converted the whole data into pandas dataframe
exceldata=pd.DataFrame(alldata)

# Finally I have done

In [61]:
# Exporting the final data into new excel file
file_name = 'Output_with_completing_tasks.csv'
exceldata.to_csv(file_name,index=False)

# Thank You